# 生成 AI による記事アイデアの作成

この Notebook では、生成 AI に記事のネタだしから調査、記事作成までを自動で実行する Agent を作成します。

PV 数の多い記事を分析して得た PV 数が高くなりそうな記事のネタの仮説を元に、Web 検索を利用し情報を集めて記事アイデアを提案するという設計にします。

## Agent の作成

まずは、 `langchain` を使用した ReACT エージェントを作成します。

In [11]:
!pip install "langchain==0.0.309" langchainhub -qU
!pip install "duckduckgo-search>=3.9.5" -qU
!pip install mecab-python3
!pip install unidic-lite

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Obtaining dependency information for mecab-python3 from https://files.pythonhosted.org/packages/16/2d/02776cf79c5961adc52fbd34f72938cf84d68be18794a19f6a428940d718/mecab_python3-1.0.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 6.9 MB/s eta 0:00:00:00:01
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached unidic_lite-1.0.8-py3-none-any.whl


In [3]:
import langchain
from langchain.llms.bedrock import Bedrock
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools.render import render_text_description
from langchain.agents import Tool, AgentExecutor
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain import hub
import boto3

# langchain.verbose = True

# Initialize Model
bedrock_client = boto3.client("bedrock-runtime")
llm = Bedrock(
    model_id="anthropic.claude-instant-v1",
    client=bedrock_client,
    model_kwargs={ 'max_tokens_to_sample': 1024 }
)

# Initialize Tool
wrapper = DuckDuckGoSearchAPIWrapper(region="jp-jp", safesearch="moderate")
search = DuckDuckGoSearchRun(api_wrapper=wrapper)
tools = [
    Tool(
        name="duckduckgo-search",
        func=search.run,
        description="日本語もしくは英語でウェブ検索が可能",
    )
]

デフォルトの `langchain` の Agent のプロンプトは GPT 用に作成されているため、`Claude` の[プロンプトベストプラクティス](https://docs.anthropic.com/claude/docs/introduction-to-prompt-design)を反映したプロンプトでテンプレートを上書きします。

In [16]:
# Define Prompt
prompt = hub.pull("hwchase17/react")
tool_names = ", ".join([t.name for t in tools])
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=tool_names,
)
print("元のプロンプト: ", prompt.template)
print("---")
prompt.template = """

Human: 以下の instruction で与えられた指示に従ってください。以下のツールを利用することが可能です。

<ツール>
{tools}
</ツール>

以下のフォーマットを使用してください:

<output-format>
Thought: 指示に答えるために何をするべきか検討
Action: 実行するアクション。必ず {tool_names} から選択する。
Action Input: Action への入力
Observation: アクションの結果
... (Thought/Action/Action Input/Observation を N 回繰り返す)
Thought: 答えるのに必要な情報が揃いました
Final Answer: instruction に対する出力
</output-format>

それでは以下から開始してください。

<instruction>
Instruction: {input}
</instruction>

Assistant:
Thought:{agent_scratchpad}
"""
print("Claude に最適化されたプロンプト: ", prompt.template)

# Stop Generation on stop token (passed to Bedrock)
llm_with_stop = llm.bind(stop=["\nObservation"])

# Create Agent
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_log_to_str(x['intermediate_steps'])
} | prompt | llm_with_stop | ReActSingleInputOutputParser()

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    return_intermediate_steps=True,
    verbose=True,
)

元のプロンプト:  Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
---
Claude に最適化されたプロンプト:  

Human: 以下の instruction で与えられた指示に従ってください。以下のツールを利用することが可能です。

<ツール>
{tools}
</ツール>

以下のフォーマットを使用してください:

<output-format>
Thought: 指示に答えるために何をするべきか検討
Action: 実行するアクション。必ず {tool_names} から選択する。
Action Input: Action への入力
Observation: アクションの結果
... (Thought/Action/Action Input/Observation を N 回繰り返す)
Thought: 答えるのに必要な情報が揃いました
Final Answer: instruction に対する出力
</output-format>

それでは以下から開始して

## 出力のハイライト

著作権の配慮および Hallucination の検知のために、出力のうち、検索で取得したデータをハイライトして表示します。

In [42]:
from IPython.display import display, HTML
import MeCab
import re

wakati = MeCab.Tagger("-Owakati")
style = """<style>
em { color: #999999; }
.c1 { color: #FF0000; }
.c2 { color: #00FF00; }
.c3 { color: #0000FF; }
.c4 { color: #FFFF00; }
.c5 { color: #FF00FF; }
.c6 { color: #00FFFF; }
.c7 { color: #CC0000; }
.c8 { color: #00CC00; }
.c9 { color: #0000CC; }
.c10 { color: #CCCC00; }
.c11 { color: #CC00CC; }
.c12 { color: #00CCCC; }
.c13 { color: #990000; }
.c14 { color: #009900; }
.c15 { color: #000099; }
.c16 { color: #999900; }
.c17 { color: #990099; }
.c18 { color: #009999; }
</style>
"""


def create_ngram(string, n=2):
    delimiter = ['「', '」', '…', '　']
    if n == 1:
        return list(string)
    elif n == 2:
        double = list(zip(string[:-1], string[1:]))
        double = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter))), double)
        return list(double)
    elif n == 3:
        triple = list(zip(string[:-2], string[1:-1], string[2:]))
        triple = filter((lambda x: not((x[0] in delimiter) or (x[1] in delimiter) or (x[2] in delimiter))), triple)
        return list(triple)
    return []


def create_ngrams(string):
    return create_ngram(string, 1) + create_ngram(string, 2) + create_ngram(string, 3)


def process_result(result):
    output = result["output"].replace("<output>", "").replace("</output>", "")
    search_results = "".join([x[1].replace("...", "\n\n") for x in result["intermediate_steps"]]).split("\n\n")

    # わかちがき -> ngram
    output_tokens = wakati.parse(output).split()
    output_ngram = set(create_ngrams(output_tokens))
    checked_ngram = set()
    
    # TODO: ハイライトの修正。Longest Match のフレーズでハイライト。
    
    for idx, search_result in enumerate(search_results):
        # わかちがき -> ngram
        search_tokens = wakati.parse(search_result).split()
        search_ngram = set(create_ngrams(search_tokens)) - checked_ngram
        checked_ngram = checked_ngram | search_ngram
        
        # Match 
        token_matches = output_ngram & search_ngram
        matches = ["".join(list(x)) for x in token_matches]
    
        # Wrap match with <em></em>
        for match in matches:
            output = output.replace(match, f'<em class="c{idx}">' + match + "</em>")

    output = re.sub(r'(<em class="c)[0-9]+("></em>)', '', output)
    output = output.replace("\n", "<br/>")
    
    return style + "<div>" + output + "</div>"

## Agent の検証

In [38]:
result = agent_executor.invoke({
    "input": "あなたは敏腕記者です。今話題の日本の芸能人で話題になりそうなものを一つピックアップし深掘りして関連情報などを調査し1500文字程度のニュース記事を作成してください。"
})
print("--- 出力 ---")
HTML(process_result(result))



> Entering new AgentExecutor chain...
芸能人で今話題の人物がいるのかウェブ検索で調べる

Action: duckduckgo-search  

Action Input: "今話題の日本の芸能人"
｜芸能｜ 2 Cocomi＆Koki,、木村拓哉と工藤静香の夫婦ショット公開 木村の誕生日を祝福「最高な1年にしちゃってくださいな」 フルート奏者でモデルのCocomi（22）が13日、自身のインスタグラムを更新。 父で俳優・木村拓哉（51）の誕生日を祝福した。 2023-11-13 10:35 ｜SNS発｜ 3 35歳・鈴木奈々「お尻上がった」... 戸愚呂兄弟役は綾野剛＆滝藤賢一、出演者のビジュアル巡り物議 0 1 7 2023年11月11日（土）PM 19:38 X JAPANがベース・HEATHの死去と死因を正式発表、メンバーの追悼コメントも公開。 追悼ライブを今後開催か 0 1 0 2023年11月11日（土）PM 17:31 木下優樹菜が全力!脱力タイムズでテレビ復帰、芸能活動再開か。 タピオカ事件もネタに利用、番組出演に批判殺到も… 0 1 12 2023年11月11日（土）PM 14:46 神田正輝が病気説否定、激ヤセの理由説明も今日の旅サラダで更に老化姿に心配の声。 動画・画像あり 0 1 6 2023年11月10日（金）PM 22:29 海老原優香アナがX Capital野原秀介社長と結婚秒読みか。 今年で開催6回目となる「文春オンライン」のお笑い芸人アンケート。テレビはもちろん、YouTubeや俳優、監督、作家業、MCからコメンテーターまで、幅広いジャンルで活躍の場を広げるお笑い芸人たちの中で、2022年に最も高い好感度を得たのははたして誰なのでしょうか？ ニュース 特集 1～20／134,974 件 1 2 3 4 5 次へ 山崎育三郎、『トッツィー』制作会見で新キャラクター (?)"ドロ三郎"爆誕 見どころは「このお尻」 俳優の山崎育三郎が13日、都内で行われたミュージカル『トッツィー』の製作発表記者会見に出席。 劇中歌の歌唱パ... 2023-11-14 05:00 エハラマサヒロ、『トッツィー』制作発表で舌好調... 年末の賞レースで盛り上がるお笑い界。 先日も『THE W』の6代目女王・天才ピア

In [39]:
result = agent_executor.invoke({
    "input": "あなたは敏腕経済記者です。直近の日本の企業ニュースを一つピックアップし深掘り調査し深掘りして関連情報などを調査し1500文字程度のニュース記事を作成してください。"
})
print("--- 出力 ---")
HTML(process_result(result))



> Entering new AgentExecutor chain...
この指示に従って記事を作成するために、日本企業の最近のニュースを調査する必要があります。

Action:duckduckgo-search
Action Input: 日本企業最近のニュース
半導体の受託生産で世界最大手の台湾のTSMCが熊本県菊陽町に新たな工場を建設すると正式に発表してから11月9日で2年となりました。. 工場では ... 企業・産業 ビッグモーター問題 中古車販売大手ビッグモーターが自動車保険の保険金不正請求を行っていたことが明らかになりました。 クレジットカード決済のシステム障害 運営会社がおわび 2023/11/11 23:45 290文字... 輸出. 海外進出. 輸入. 外国企業誘致. 知的財産権保護. ジェトロが世界中から収集した各国・地域の政治・経済動向、貿易・投資に関する制度、統計、市場動向などの最新ビジネスニュースを発信しています。. ジェトロ・メンバーズ（会員）の方は記事詳細 ... 産経新聞社のニュースサイト。 産業・ビジネス（製造業、自動車、サービス業、小売り、食品、鉄道、企業経営など）に関する速報記事と解説記事を掲載しています。 日本経済新聞の電子版。日経や日経BPの提供する経済、企業、国際、政治、マーケット、情報・通信、社会など各分野のニュース。ビジネス、マネー、IT、スポーツ、住宅、キャリアなどの専門情報も満載。
TSMCの新工場建設計画が2年に関連している部分が深掘りできそうです。

Action: duckduckgo-search 
Action Input: TSMC 新工場建設 熊本
半導体の受託生産で世界最大手の台湾のTSMCが熊本県菊陽町に新たな工場を建設すると正式に発表してから11月9日で2年となりました。. 工場では ... 半導体受託製造の世界最大手、台湾積体電路製造（TSMC）が熊本県菊陽町で建設を進めてきた新工場の工事が大詰めを迎えている。2024年末を予定 ... また、熊本県にとっては大きな経済効果が見込まれていて、地元の金融機関「九州フィナンシャルグループ」は、tsmcの工場への設備投資に加えて ... 歓迎TSMC従業員. シリコンアイランド. 半導体受託生産の世界最大手、台湾積体電路製造（TSMC）が熊本

## 記事アイデアの作成

Agent がインターネットから必要な情報を検索して記事を書く検証ができたため、次は PV 数が高い記事を分析した結果をもとに、ヒットする記事の仮説を立てて必要な情報を収集し記事の草案を作成させます。

In [40]:
import pandas as pd
import os

output_path = "analysis.csv"
if os.path.exists(output_path):
    hits = pd.read_csv(output_path)
    for idx, row in hits.iterrows():
        print(row["analysis"])
        print("")

- クイズ形式の記事は読者の知識を試す面白さがある
- テレビ番組に関するクイズは、多くの人が親しみを感じる題材
- ジェパディの実際の問題を使っていることで、クイズの難易度と面白さが保証される
- 記事タイトルがクイズ形式で、読者の興味を引く
- 記事の概要で、クイズの形式と内容が明確に説明されている
- 同じ形式と題材のクイズ記事を定期的に提供することで、読者を獲得できる

- タイトルに「最も」「全米No.1」といった強調表現を用い、読者の興味を引く
- 高級ホテルという非日常的な体験に対する人々の憧れをテーマにする
- セレブリティが訪れるホテルという著名人との接点をアピールする
- 掲載ホテルの写真を多用し、非日常的な雰囲気を伝える 
- SNSでのシェア数をアピールすることで話題性を高める

- 「あらゆるタイプの人」「あらゆる予算」というキーワードがあるので、多くの人に関心のある普遍的なテーマだと思われる。
- 「驚くべき」「完璧な」といった言葉でクリック率を高めている。
- 誕生日やクリスマスなどの贈り物選びのシーズンに合わせたタイミングの良い記事。
- Reviewedという信頼できる情報源からの贈り物のおすすめ記事。
- 具体的な商品のおすすめがあるため、実際に購入に結びつきやすい。

- セレブリティの金銭トラブルは注目を集める話題だ。セレブの破産や巨額の報酬、資産など金銭に関するゴシップ記事を作ることでPVを獲得できる。
- 過去10年間の出来事をまとめたリスト形式は読みやすくPVを集めやすい。時代背景を踏まえた10年間のトレンドを分析した記事を作れば再現できる。
- 著名人のスキャンダルやゴシップは関心が高く、クリック率が良い。セレブの金銭問題を扱った記事を作ればPVを獲得できる。

- 有名人の軍歴に関する記事であるため、有名人の過去の秘話や意外な一面に関心がある読者を引きつけている
- 具体的な有名人の名前が記載されているため、その有名人のファンも記事に興味を持つ
- 軍隊経験は一般人にはなじみが薄い世界なので、著名人のそうした経験に興味が惹かれる
- 同様の著名人の過去のエピソードや意外な経歴を扱った記事を制作することで、PVを再現できる可能性がある

- アメリカの上院議員に関する記事であるため、アメリカの政治に関心のある読者を引き

In [41]:
for idx, row in hits[:3].iterrows():
    print("")
    print("##########################")
    print("--- PV 数の多い記事の要素 ---")
    print(row["analysis"])
    result = agent_executor.invoke({
        "input": f"""あなたは記者です。PV 数の多い記事の要素をもとに記事のアイデアを作成し、必要な情報を調査し記事の原稿としてまとめあげて報告してください。
<PV 数の多い記事の要素>
{row["analysis"]}
</PV 数の多い記事の要素>"""
    })
    print("--- 出力 ---")
    display(HTML(process_result(result)))


##########################
--- PV 数の多い記事の要素 ---
- クイズ形式の記事は読者の知識を試す面白さがある
- テレビ番組に関するクイズは、多くの人が親しみを感じる題材
- ジェパディの実際の問題を使っていることで、クイズの難易度と面白さが保証される
- 記事タイトルがクイズ形式で、読者の興味を引く
- 記事の概要で、クイズの形式と内容が明確に説明されている
- 同じ形式と題材のクイズ記事を定期的に提供することで、読者を獲得できる


> Entering new AgentExecutor chain...
記者として、PV数の多い記事の要素をもとにクイズ形式の記事のアイデアを考える必要がある。  

Action: duckduckgo-search 

Action Input: PVの高いクイズ記事の要素 
本記事では、PV数を増やすために知っておきたい基礎知識や、PVが向上しない原因PV数を増やすためのコンテンツ制作におけるテクニックなどを紹介します。 ぜひ参考にしてみてください。 関連記事： SXO対策とは？ SEOとの違いや実施のメリット、具体的な方法を解説! クリックできる目次 PVを向上させるために知っておきたい基礎知識 コンテンツのPVが増えない原因 PVを向上させるコンテンツ制作のテクニック PVを増やす際に必要な分析の方法 PV向上を狙うなら記事制作代行を利用するのもおすすめ 記事制作代行を利用するメリット まとめ PVを向上させるために知っておきたい基礎知識 PV数を向上させる施策を行う前に、知っておきたい基礎知識が2つあります。 ここでは、それぞれ詳しく見ていきましょう。 目次 1 PV数とは 2 セッション数との違い 3 オウンドメディアの平均的なPV数の目安は？ 4 オウンドメディアのPV数の目標（KPI）を設定する方法 5 オウンドメディアのPV数で重要な3つの指針 6 オウンドメディアのPV数を引き上げる方法 7 オウンドメディアの効果測定と改善案 8 まとめ：オウンドメディアで目標PV数を達成させるために PV数とは PV数とは、ページビュー数の略称で、 Webサイトやブログなどのアクセス数を測る重要な指標の一つ です。 PV数は、あるユーザーが特定のWebサイトのページを何回表示したか


##########################
--- PV 数の多い記事の要素 ---
- タイトルに「最も」「全米No.1」といった強調表現を用い、読者の興味を引く
- 高級ホテルという非日常的な体験に対する人々の憧れをテーマにする
- セレブリティが訪れるホテルという著名人との接点をアピールする
- 掲載ホテルの写真を多用し、非日常的な雰囲気を伝える 
- SNSでのシェア数をアピールすることで話題性を高める


> Entering new AgentExecutor chain...
Instruction から記事作成のアイデアと必要な情報を調査するための基盤情報を得られそうな要素が伝えられている

Action: duckduckgo-search 
Action Input: 最高級ホテルランキング
NEW 旅行シーズン到来! 冬の宿泊先選び 大阪 【大阪】高級ホテル ベスト 沖縄 【沖縄】高級ホテル ベスト 東京 【東京】高級ホテル ベスト 大阪 【大阪】The ベストホテル 名古屋 【名古屋】高級ホテル ベスト 福岡 【福岡】高級ホテル ベスト 大阪 【大阪】USJホテル ベスト 東京 【東京】ディズニーホテル ベスト 京都 【京都】高級ホテル ベスト 大阪 【大阪】高級ホテル ベスト 沖縄 【沖縄】高級ホテル ベスト 東京 【東京】高級ホテル ベスト 大阪 【大阪】The ベストホテル 名古屋 【名古屋】高級ホテル ベスト 福岡 【福岡】高級ホテル ベスト 大阪 【大阪】USJホテル ベスト Prev Next 国内ホテル 東京 今回は、極上体験ができると定評のある日本のラグジュアリーホテルランキングTOP10をご紹介します。 ぜひ、記念日に大切な人と過ごすホテル選びの参考にしてみてください。 ※コロナ禍での旅行における注意点 新型コロナウイルスの感染拡大防止のため、ホテルの営業時間・サービス内容を変更していることがあります。 ご予約の際は公式HPまたはお電話でご確認ください。 https://www.jalan.net/ 日本のホテルのクオリティは？ 旅行サイトで訪日した海外からの観光客の口コミを見ると、「日本のホテルはサービスが素晴らしい。 」「心のこもったホスピタリティー」という声が多くみられます。 TOP おすすめ特集 高級ホテ


##########################
--- PV 数の多い記事の要素 ---
- 「あらゆるタイプの人」「あらゆる予算」というキーワードがあるので、多くの人に関心のある普遍的なテーマだと思われる。
- 「驚くべき」「完璧な」といった言葉でクリック率を高めている。
- 誕生日やクリスマスなどの贈り物選びのシーズンに合わせたタイミングの良い記事。
- Reviewedという信頼できる情報源からの贈り物のおすすめ記事。
- 具体的な商品のおすすめがあるため、実際に購入に結びつきやすい。


> Entering new AgentExecutor chain...
記事のアイデを考えPV数が多い要素を取り入れる必要がある。

Action: duckduckgo-search
Action Input: "あらゆるタイプの人" "あらゆる予算" 贈り物のアイデア

普遍的なテーマである"あらゆるタイプの人"と"あらゆる予算"の贈り物アイデアに合わせた記事とすることが良いだろう。

Action: duckduckgo-search 
Action Input: Reviewed 「驚くべき」「完璧な」贈り物
季節の変わり目や人生の節目に贈るギフト。食べ物やお菓子、おしゃれなものや堅苦しくないものなど、種類を決めるのも難しいです。さらに、親戚や上司など相手によってはどんなものが適切なのか迷ってしまいます。そこで今回は贈り物の選び方やおすすめ商品を人気ランキング形式で紹介 ... 想い出に残るサプライズを演出しよう | 伝統工芸品ならBECOS. 還暦祝いの変わったプレゼント27選!. 想い出に残るサプライズを演出しよう. 2023年7月10日. ライターダル. 還暦は一生に一度しかなく、定年退職と重なることもあるため、人生の節目となるお祝い ... 1000円以内で用意するちょっとしたお礼やプチギフトには、自分で買わないけどもらって嬉しいものが理想です。今回は贅沢過ぎないおしゃれなブランドものから面白いネタ系のプレゼントまで、男性女性問わず、学生から大人まで喜ばれる選び方のポイントをご紹介します。 1万円以下で購入できるプレゼントでおすすめな高級日用品8選. ここからは、予算1万円以下で購入できるプレゼントにおすすめの高級品をご紹介します。. 毎日使える